<a href="https://colab.research.google.com/github/Rohit-Saswadkar/Font-Text-classification-project-using-CNN/blob/main/2%5D_Font_Recognition_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Font Recognition App using Streamlit

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Technologies used -
* Open cv - to image processing
* easy ocr - to detect texts and regions

In [37]:
!pip install streamlit
!pip install easyocr

In [39]:
import streamlit as st
import cv2
import easyocr
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image

Overwriting app.py


In [42]:
# List of font class names
classes = ['Agency', 'Akzidenz Grotesk', 'Algerian', 'Arial', 'Baskerville', 'Bell MT', 'Bembo', 'Bodoni', 'Book Antiqua', 'Brandish', 'Calibry', 'Californian FB', 'Calligraphy', 'Calvin', 'Cambria', 'Candara', 'Century', 'Comic Sans MS', 'Consolas', 'Corbel', 'Courier', 'Didot', 'Elephant', 'Fascinate', 'Franklin Gothic', 'Frutiger', 'Futura', 'Garamond', 'Georgia', 'Gill sans', 'Helvetica', 'Hombre', 'LCD Mono', 'Lucida Bright', 'Minion', 'Monotype Corsiva', 'Mrs Eaves', 'Myriad', 'Nasalization', 'News Gothic', 'Palatino linotype', 'Perpetua', 'Rockwell', 'Sabon', 'Snowdrift Regular', 'Steppes TT', 'Times New Roman', 'Verdana']


* 1] Load pre-trained model

In [43]:
# Define path to our font recognition model
font_model_path = r"/content/drive/MyDrive/Data Science Advanced Projects/1 Data Science Advanced/Deep learning/Font recognition assessment/Models/Font Dataset Large Model H5 file new 10.h5"


In [48]:
# load our pretrained font recognition model
font_model = load_model(font_model_path)

2] Data preprocessing
* Converted color image to grey scale using open cv
* Resizing the image to 64 x 64 using open cv
* Normalizing the values of the image in the range of 0 to 255.

In [46]:
# Function to preprocess the image patch for your font recognition model
def preprocess_image(image_patch):
    # convert to grayscale as our model trained on grayscale
    gray_patch = cv2.cvtColor(image_patch, cv2.COLOR_BGR2GRAY)
    # resize the image patch to a fixed size expected by your model
    resized_patch = cv2.resize(gray_patch, (128, 32))
    # normalize pixel values (convert to range 0-1)
    normalized_patch = resized_patch / 255.0
    return normalized_patch


3] created a image font prediction function using my model
* it get input as image patch , model and classes of the dataset model trained on and returns the class of detected font

In [47]:
# my function to predict font using your model
def predict_font(image_patch, font_model, classes):
    # preprocess each patch
    preprocessed_patch = preprocess_image(image_patch)

    predicted_font = font_model.predict(np.expand_dims(preprocessed_patch, axis=0)) # ------------------- our model for font prediction

    predicted_font_index = np.argmax(predicted_font)

    predicted_font_name = classes[predicted_font_index]

    return predicted_font_name


In [49]:
# Streamlit app interface
st.title("Font Recognition App")
st.write('by Rohit Rajendra Saswadkar')
st.write("Upload an image, and the app will detect text regions and recognize the fonts used.")

In [50]:
# File uploader for the image
uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

* This extracts the each box with its text and confidence score
* If confidence score > 0.4 then it shows the text else not.
* The line 1) creates text regions on the image using open cv
* Theline 2) shows the extracted texts with predicted font on the image using open cv
* Finally this line 3) shows the image in colors

In [51]:
if uploaded_file is not None:
    # Image reading
    # Reading the images using opencv
    # Used Optical Character Recognition (easy OCR) to detect and extract texts from image and create text bounding boxes
    image = Image.open(uploaded_file)
    img = np.array(image)

    # Text detection with EasyOCR
    reader = easyocr.Reader(['en'], gpu=False)
    text_ = reader.readtext(img)

    threshold = 0.25

    # Draw bbox, text, and predicted font
    for t_, t in enumerate(text_):
        bbox, text, score = t
        if score > threshold:
            # Draw the bounding box
            cv2.rectangle(img, tuple(bbox[0]), tuple(bbox[2]), (0, 255, 0), 2) # --------------------------------------------- 1) to show region bounded texts

            # Extract the text patch
            x_min = min(bbox[0][0], bbox[3][0])
            x_max = max(bbox[1][0], bbox[2][0])
            y_min = min(bbox[0][1], bbox[1][1])
            y_max = max(bbox[2][1], bbox[3][1])
            text_patch = img[y_min:y_max, x_min:x_max]

            if text_patch.size > 0:
                # Predict font using your model
                predicted_font = predict_font(text_patch, font_model, classes) # ----------extracted text patch, model and classes are forword to prediction function and it detects

                # Display text and predicted font
                font_text = f"{text} (Font: {predicted_font})"
                cv2.putText(img, font_text, (bbox[0][0], bbox[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2) # ---2) to show texts and predicted font


Display the input and processed image

In [ ]:
 # Display the images side by side using Streamlit columns
    col1, col2 = st.columns(2)
    with col1:
        st.header("Input Image")
        st.image(image, caption='Uploaded Image', use_column_width=True)

    with col2:
        st.header("Processed Image")
        st.image(img, caption='Processed Image', use_column_width=True)